In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, db
import json
import sys
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import os
from tqdm import tqdm

####### select task name and version to download #######
taskname = 'rew-eff-ema'
version = 'study1'
project_dir=(r'C:/Users/shewitt/Documents/GitHub/reward-effort-2afc-firebase-BEX-FU/')
data_dir=(r'D:/EMA_Motivation/data/study1/ema/task/')
######
isExist = os.path.exists(data_dir)
if not isExist:
   # Create data_dir because it does not exist
    os.mkdir(data_dir)

# set up firebase credentials
cred = credentials.Certificate('D:/EMA_Motivation/data/study1/firebase_private_key/ema-motivation-firebase-adminsdk-1tmnj-a042345e48.json')
default_app = firebase_admin.initialize_app(cred)
print("Firebase connected")

Firebase connected


In [2]:
# get all current data
client = firestore.client(default_app)
# make a dir for raw data
isExist = os.path.exists((data_dir + 'raw/'))
if not isExist:
   # Create data_dir because it does not exist
    os.mkdir(data_dir + 'raw/')

# lists to save
practice=[]; pre=[]; task=[]

# Initialize a counter and calculate the maximum dynamically
downloaded_count = 0
error_count = 0
total_files = sum(1 for _ in client.collection(taskname).document(version).collections() for _ in _.stream())

# loop across all current participants
collections = client.collection(taskname).document(version).collections()
with tqdm(total=total_files, unit="file") as pbar:
    print('downloading files...')
    for sub in collections:
        for attempt in sub.stream():
            #skip the maxthreshold documents
            try:
                if attempt.id!="maxThreshold":
                    # get id info:
                    idinfo = client.document(taskname+'/'+version+'/{0}/{1}'.format(sub.id, attempt.id)).get().to_dict()
                    subid = str(idinfo['userHash'])
                    attemptID = str(idinfo["attemptHash"])
                    fsid = str(idinfo['firebaseUID'])  
                    # if this participant at least started the task, get their data 
                        # get practice task data, pre-task questions and main task data
                    # there's no practice for follow-up short game attempts:
                    if str(idinfo["gameType"])=="baseline":
                        pracdata = client.document(taskname+'/'+version+'/{0}/{1}/practice-data/data'.format(sub.id, attempt.id)).get().to_dict()
                        # write data to the raw directory
                        with open(data_dir + 'raw/' + version+'-'+subid+'-'+fsid+'-'+attemptID+'-prac-data.txt','w') as f:
                            f.write(json.dumps({ **idinfo, **pracdata})) # practice task data
                    
                    # for other follow up games:
                    qdata = client.document(taskname+'/'+version+'/{0}/{1}/quest/data'.format(sub.id, attempt.id)).get().to_dict()
                    taskdata = client.document(taskname+'/'+version+'/{0}/{1}/task-data/data'.format(sub.id, attempt.id)).get().to_dict()

                    with open(data_dir + 'raw/' + version+'-'+subid+'-'+fsid+'-'+attemptID+'-quest.txt','w') as f:
                        f.write(json.dumps({ **idinfo, **qdata})) # STATE Qs 
                    with open(data_dir +'raw/' + version+'-'+subid+'-'+fsid+'-'+attemptID+'-task-data.txt','w') as f:
                        f.write(json.dumps({ **idinfo, **taskdata})) # TASK data
                    #print("id:"+subid+' ver:'+version+' attempt:'+attemptID+ ' downloaded!')

                    # Update the counter
                    downloaded_count += 1

                    # Update the progress bar
                    pbar.update(1)
            except:
                # update counter
                error_count += 1

                # if you want, print the error ids:
                #print("id:"+subid+' ver:'+version+' attempt:'+attemptID+ ' error!')
                #print('error uid = '+subid)
                #print('attempt hash = '+attemptID)
 
# end messages:
print('all done.')
# Print the total number of errors
print(f"Total errors: {error_count}")

  0%|          | 0/1219 [00:00<?, ?file/s]

downloading files...


 85%|████████▍ | 1034/1219 [01:35<00:17, 10.83file/s]

all done.
Total errors: 53
